# step epoch iteration

| 용어 | 의미 |
|---|---|
| Epoch | 전체 학습 데이터셋을 한 번 모두 모델에 학습시키는 주기입니다. |
| Iteration | 한 번의 미니 배치(batch)를 모델에 학습시키는 단위입니다. |
| Step | 일반적으로 iteration과 동일한 의미로 사용됩니다. (특히 PyTorch에서) |


전체 데이터셋에 샘플이 1,000개 있다고 가정하겠습니다.

한 번에 학습에 사용하는 배치 크기(batch_size)는 100개라고 하겠습니다.

이 경우,

1 Epoch = 전체 데이터 1,000개를 다 학습   
→ 100개씩 10번 반복해야 하므로   
→ 10 Iterations (or 10 Steps)

---

다음 식은 참고로만 알아두죠!

$\Large step = \frac{N\times \text{epoch}}{\text{\# of GPU} \times \text{batch size} \times \text{gradient accumulation}}$

# Regularization
> 모델이 훈련 데이터에만 **과하게 적합(overfitting) 되는 것을 막기 위해**, 모델의 복잡도를 제한하는 방법

흔히 시련이 있으면 더 성장한다고 하죠   
학습할 때 모델에 시련을 줘서 더 성능이 좋아지도록 합니다 (특히 테스트 데이터에서 성능)

| 종류                | 설명                                                     |
| ----------------- | ------------------------------------------------------ |
| **L1 규제 (Lasso)** | 가중치들의 절댓값 합에 패널티를 줍니다. → **희소한 모델(많은 가중치를 0으로)** 만듭니다. |
| **L2 규제 (Ridge)** | 가중치들의 제곱합에 패널티를 줍니다. → **큰 가중치를 작게 줄이는 역할**을 합니다.      |
| **Dropout**       | 학습 시 일부 뉴런을 무작위로 꺼서 특정 뉴런에 의존하지 않게 합니다.                |


# EMA(Exponential Moving Average, 지수 이동 평균)
> 지수 이동 평균(EMA) 은 최근 값을 더 중요하게 반영하며, 과거 값을 점점 덜 반영하는 가중 평균

이번에 배울 배치 정규화(batch normalization)에서 running mean, running var를 추정할 때 나옵니다!   
말 그대로 추정값이라 실제 값과 오차가 있을 수 있습니다.

$\Large \mu_t = \alpha x_t + (1-\alpha)\mu_{t-1}$

- $x_t$: 현재 배치에서 측정된 통계값 (예: 배치 평균, 분산 등)
- $\mu_{t-1}$: 이전까지의 EMA
- $\alpha$: decay 계수 (0~1 사이)

torch 예제를 봅시다

In [18]:
import torch

num_samples: int = 10000

# 전체 데이터 생성
data: torch.Tensor = torch.randn(num_samples)

print(data)

# 전체 데이터 기준 정답 통계
true_mean = data.mean()

# 출력
print(f"평균: {true_mean}")

tensor([-1.2896, -0.6921, -1.4356,  ..., -0.9725, -1.2570,  1.5115])
평균: 0.0021140272729098797


In [19]:
# EMA 파라미터 초기화
ema = 0
momentum = 0.001 

batch_size = 5

# 배치 단위로 EMA 업데이트
for i in range(0, num_samples, batch_size):
    mini_batch = data[i:i+batch_size]
    ema = momentum * ema + (1 - momentum) * mini_batch.mean()

print(f"EMA: {ema}")

EMA: 0.0026559513062238693


차이가 있지만 평균이랑 EMA랑 비슷하네요!